% 'account.login' on various platforms

In [1]:
%matplotlib inline

In [1]:
TOTAL_QUERY = "SELECT C4 FROM %s WHERE C5 = 'account.login'"

In [1]:
CHUNK_QUERIES = [('Android', "SELECT C4 FROM %s WHERE C5 = 'account.login' AND C3 = 'Android'"), ('Desktop', "SELECT C4 FROM %s WHERE C5 = 'account.login' AND (C3 = 'Linux' OR C3 = 'Macintosh' OR C3 = 'Windows 7' OR C3 = 'Windows 8' OR C3 = 'Windows 8.1' OR C3 = 'Windows Vista' OR C3 = 'Windows XP' OR C3 = 'Windows 10')"), ('Firefox for iOS', "SELECT C4 FROM %s WHERE C5 = 'account.login' AND C1 = 'FxiOS'")]

In [1]:
TITLE = "% 'account.login' on various platforms"

In [1]:
EVENT_STORAGE = "s3n://net-mozaws-prod-us-west-2-pipeline-analysis/fxa-retention/data/"

In [1]:
import os
from datetime import date, timedelta, datetime
import pandas

IN_IPYTHON = True
try:
    __IPYTHON__
except NameError:
    IN_IPYTHON = False

today = date.today()
# today = datetime.strptime('2015-11-08', '%Y-%m-%d').date()
last_monday = today - timedelta(days=-today.weekday(), weeks=1)

WEEK_RANGE = pandas.date_range(end=last_monday, periods=15, freq='W-MON')
WEEKS = WEEK_RANGE.map(lambda x: x.strftime('%Y-%m-%d'))

if not IN_IPYTHON:
    EVENT_STORAGE = os.path.join(os.path.dirname(os.path.abspath(__file__)), '..', '..', 'tools', 'out')

OUT_DATA = []
VOLUME_DATA = []

for v in range(0, len(WEEKS)):
    VOLUME_DATA.append([0] * len(WEEKS))

for x in range(0, len(WEEKS)):
    OUT_DATA.append([0] * len(WEEKS))

def week_file(storage, week):
    return os.path.join(storage, 'events-' + week + '.csv')


In [1]:
import pandas, os, sys

# Only initialize Spark if testing locally
# Otherwise it should be already running within Spark
try:
    from pyspark import SparkContext
except ImportError:
    import dev_env

from pyspark import SparkContext
from pyspark.sql import SQLContext, Row

IN_IPYTHON = True

try:
    __IPYTHON__
except NameError:
    IN_IPYTHON = False
    sc = SparkContext('local')
    sys.path.append(os.path.realpath(os.curdir))
    from ipynb_generators.parts.weeks import WEEKS, WEEK_RANGE, OUT_DATA, VOLUME_DATA, EVENT_STORAGE, week_file
    TOTAL_QUERY = "SELECT C4 FROM %s WHERE C5 = 'account.created'"
    CHUNK_QUERIES = [
        ("Android", TOTAL_QUERY + " AND C3 = 'Android'"),
        ("Firefox for iOS", TOTAL_QUERY + " AND C1 = 'FxiOS'")
    ]
    print "Not in IPython, creating SparkContext manually, using fake data!"

# sc will be global in IPython
sqlContext = SQLContext(sc)

WEEKLY_DATA = {}


for week in WEEKS:
    df = sqlContext.load(source='com.databricks.spark.csv', header='false', path=week_file(EVENT_STORAGE, week))
    table_name = 'week' + week.replace('-', '_')
    df.registerTempTable(table_name)

    total_events = sqlContext.sql(TOTAL_QUERY % table_name)
    total_mapped_events = total_events.map(lambda p: p.C4)
    total_count = total_mapped_events.count()

    for query in CHUNK_QUERIES:
        chunk_name = query[0]
        sql_query = query[1]

        chunk_events = sqlContext.sql(sql_query % table_name)
        chunk_mapped_events = chunk_events.map(lambda p: p.C4)
        chunk_count = chunk_mapped_events.count()

        if chunk_mapped_events > 0:
            percentage = int((float(chunk_count) / float(total_count)) * 100)
        else:
            percentage = 0

        WEEKLY_DATA.setdefault(chunk_name, []).append(percentage)

print WEEKLY_DATA

In [1]:
import seaborn
import matplotlib.pyplot as plt

IN_IPYTHON = True
try:
    __IPYTHON__
except NameError:
    IN_IPYTHON = False

if IN_IPYTHON:
    if "DATA_FRAME" in locals():
        seaborn.set(style='white')
        plt.figure(figsize=(16, 12))
        plt.title(TITLE, { 'fontsize': 26 })
        seaborn.heatmap(DATA_FRAME, annot=True, fmt='d', yticklabels=WEEKS, xticklabels=range(0, len(WEEKS)))
        # Rotate labels
        locs, labels = plt.yticks()
        plt.setp(labels, rotation=0)
        # Set axis font
        font = {
            'weight': 'bold',
            'size': 22
        }
        # Label axis
        plt.ylabel('Cohort Starting Week', **font)
        plt.xlabel('Retention Weeks', **font)

        print VOLUME_DATA_FRAME

    if "WEEKLY_DATA" in locals():
            print WEEKLY_DATA
            fig = plt.figure(figsize=(16, 12))

            plt.title(TITLE, { 'fontsize': 26 })
            for item in WEEKLY_DATA:
                val = WEEKLY_DATA[item]
                plt.plot(val)
                ax = fig.add_subplot(111)
                for x,y in zip(range(15), val):
                    ax.annotate('%s' %y, xy=(x,y), size=15)

            plt.ylabel('Percentage')
            plt.setp(plt.gca().get_xticklabels(), rotation=45, horizontalalignment='right')
            plt.xticks(range(15), WEEKS)
            plt.legend(WEEKLY_DATA.keys(), loc='lower left', prop={'size': 16})


            plt.ylim(-10, 110)
            plt.show()
